In [47]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [48]:
## load the model
model = load_model('model.h5')

## load the encoder and scaler
with open('label_encoder.pkl' , 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl' , 'rb') as file:
    scaler = pickle.load(file)

with open('onehot.pkl' , 'rb') as file:
    label_encoder_geo = pickle.load(file)


In [49]:

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}


In [50]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded , columns = label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\aryan\OneDrive\Desktop\ann classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [51]:
input_df = pd.DataFrame([input_data])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [52]:
input_data = pd.concat([input_df.reset_index(drop=True) , geo_encoded_df],axis=1)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [53]:
input_data = input_data.drop('Geography', axis=1)
input_data


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [54]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [56]:
input_data_scaled = scaler.transform(input_data)
input_data_scaled

array([[-0.49786927,  0.94280904,  0.08403616, -0.73747621, -0.2781524 ,
         0.80766392,  0.67088803,  0.98632627, -0.8689603 ,  1.06066017,
        -0.59903551, -0.60096459]])

In [57]:
##predict churn
prediction = model.predict(input_data_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step


array([[0.03085057]], dtype=float32)

In [59]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.030850574)

In [60]:
if(prediction_prob>0.5):
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')


The customer is not likely to churn
